In [2]:
import pandas
from nltk import word_tokenize
from gensim.models import Word2Vec

/var/folders/sh/9y2x7hyx76bghklsygtm6m000000gn/T/ipykernel_79007/35827986.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas


In [3]:
twitter_df = pandas.read_csv('processed_cyberbullying_tweets.csv')
tweets = twitter_df['processed_tweet_text'].astype(str).tolist()

In [4]:
tokenized_sentences = [word_tokenize(sentence.lower()) for sentence in tweets]
model = Word2Vec(sentences=tokenized_sentences, vector_size=100, window=5, min_count=1, workers=4)

In [5]:
model.wv['food']

array([-0.11656152,  0.37438858,  0.3292683 ,  0.10302036,  0.24794626,
       -0.6317315 , -0.12931946,  0.6597121 , -0.26665574, -0.6152894 ,
       -0.48683625, -0.5928095 ,  0.09547028,  0.57950747,  0.33780646,
       -0.38385776,  0.7272408 , -1.1879708 , -0.03241641, -0.37247065,
        0.15060964,  0.45045048, -0.20282362, -0.00388904, -0.2818553 ,
        0.9146622 , -0.43120676,  0.31137404, -0.6128463 , -0.6574089 ,
        0.46194103,  0.3514842 , -0.6977897 , -0.07252809,  0.05028884,
        0.05093319,  0.5308221 ,  0.01195968, -0.15001433, -0.32282862,
       -0.2746927 ,  0.02712599,  0.59781885, -0.98294216, -0.0636916 ,
       -0.39662546, -0.6538947 ,  0.17041467,  0.550579  ,  0.27867228,
        0.14114442, -0.16514648, -0.056178  , -0.3345168 , -0.4168285 ,
       -0.05789689,  0.8499715 ,  0.0343224 , -0.14778924, -0.001401  ,
       -0.2966222 , -0.33313206, -0.01285759,  0.4332352 , -1.1464317 ,
        0.1906514 ,  0.26931182, -0.34804922,  0.05302043,  1.36

In [6]:
vocab_size = len(model.wv.key_to_index)
vocab_size

52456

In [7]:
twitter_df['word_embeddings'] = twitter_df.apply(lambda x : list(), axis=1)

for index, row in twitter_df.iterrows():
    for word in word_tokenize(str(row['processed_tweet_text']).lower()):
        row['word_embeddings'].append(model.wv[word])
    while len(row['word_embeddings']) < 50:
        row['word_embeddings'].append([0]*100)
    

In [8]:
len(twitter_df.loc[0, 'word_embeddings'])

50

In [9]:
len(twitter_df.loc[0, 'word_embeddings'][0])

100

In [10]:
model.wv['words']

array([ 0.03375451,  0.01639242,  0.26653334, -0.15941752, -0.02850819,
       -0.53289413,  0.06161923,  0.55192417,  0.04856513, -1.0466564 ,
       -0.15115912, -1.2139022 ,  0.04472702,  0.23184012,  0.4745724 ,
        0.033797  ,  0.39826915, -1.1947025 , -0.16768028, -0.7398385 ,
        0.18270423,  0.37173277,  0.11691167, -0.3011429 ,  0.03884634,
        1.1098007 , -0.67948616,  0.48499182, -0.5553805 , -0.46493334,
        0.3885458 ,  0.28694513, -0.17727672, -0.15302575,  0.03137971,
        0.30592123,  0.5192671 , -0.15145816, -0.06599976, -0.46803582,
       -0.5047994 , -0.10696647,  0.50924563, -0.9958814 ,  0.15856008,
       -0.5104042 , -0.27912298,  0.32230994,  0.71959764,  0.635307  ,
        0.41909635, -0.32195383,  0.07035705, -0.07057319, -0.5673716 ,
       -0.10604136,  0.8826875 , -0.02792183, -0.6713345 , -0.19214663,
       -0.34260878, -0.01082203,  0.41187453,  0.8668792 , -0.940642  ,
        0.13074401,  0.33475482,  0.20683964, -0.25828996,  1.46

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X = twitter_df['word_embeddings'].tolist()
y = twitter_df['cyberbullying_type'].tolist()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense

In [14]:
# Define the model
bilstm = Sequential()

# Embedding layer
bilstm.add(Embedding(input_dim=52456, output_dim=100, input_length=50))

# Bidirectional LSTM layer
bilstm.add(Bidirectional(LSTM(64)))

# Additional Dense layer for more complex transformations
bilstm.add(Dense(64, activation='relu'))  # Added intermediate Dense layer with 64 units

# Output layer for binary classification
bilstm.add(Dense(1, activation='sigmoid'))

# Compile the model
bilstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Model summary to see the updated architecture
bilstm.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 100)           5245600   
                                                                 
 bidirectional (Bidirection  (None, 128)               84480     
 al)                                                             
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5338401 (20.36 MB)
Trainable params: 5338401 (20.36 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
bilstm.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=1)